## Notebook demonstrating how to access behavior and ophys data from lims or NWB files for Visual Behavior platform paper analysis

Use with VBA dev branch and AllenSDK master branch (2.8.0)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
import visual_behavior.data_access.loading as loading

## Get ophys experiments that are in the March release

Using VBA <code>get_filtered_ophys_experiment_table()</code> with <code>release_data_only</code> flag set to True

In [4]:
help(loading.get_filtered_ophys_experiment_table)

Help on function get_filtered_ophys_experiment_table in module visual_behavior.data_access.loading:

get_filtered_ophys_experiment_table(include_failed_data=False, release_data_only=False, exclude_ai94=True)
    Loads a list of available ophys experiments and adds additional useful columns to the table. By default, loads from a saved cached file.
    If cached file does not exist, loads list of available experiments directly from lims using SDK BehaviorProjectCache, and saves the reformatted table to the default Visual Behavior data cache location.
    
    Keyword Arguments:
    
        include_failed_data {bool} -- If True, return all experiments including those from failed containers and receptive field mapping experiments.
                                      If False, returns only experiments that have passed experiment level QC.
    
        release_data_only {bool} -- If True, return only experiments that have been reprocessed, passed experiment and container level QC, and are

In [5]:
experiments = loading.get_filtered_ophys_experiment_table(release_data_only=True)
print('there are', len(experiments), 'experiments in the release')

there are 1109 experiments in the release


In [6]:
experiments.head()

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,...,super_container_id,cre_line,session_tags,failure_tags,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,model_outputs_available,location,session_number
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
951980471,951410079,951520319,1018028342,VisualBehaviorMultiscope,published,passed,20190920_457841_1imagesA_2,OPHYS_1_images_A,MESO.1,2019-09-20 09:45:29.897856,...,850862430,Sst-IRES-Cre,NaN,NaN,0,65.0,0.0,False,Sst_VISp_150,1
951980475,951410079,951520319,1018028339,VisualBehaviorMultiscope,published,passed,20190920_457841_1imagesA_2,OPHYS_1_images_A,MESO.1,2019-09-20 09:45:29.897856,...,850862430,Sst-IRES-Cre,NaN,NaN,0,65.0,0.0,False,Sst_VISp_75,1
951980486,951410079,951520319,1018028360,VisualBehaviorMultiscope,published,passed,20190920_457841_1imagesA_2,OPHYS_1_images_A,MESO.1,2019-09-20 09:45:29.897856,...,850862430,Sst-IRES-Cre,NaN,NaN,0,65.0,0.0,False,Sst_VISl_300,1
951980479,951410079,951520319,1018028354,VisualBehaviorMultiscope,published,passed,20190920_457841_1imagesA_2,OPHYS_1_images_A,MESO.1,2019-09-20 09:45:29.897856,...,850862430,Sst-IRES-Cre,NaN,NaN,0,65.0,0.0,False,Sst_VISl_150,1
951980481,951410079,951520319,1018028357,VisualBehaviorMultiscope,published,passed,20190920_457841_1imagesA_2,OPHYS_1_images_A,MESO.1,2019-09-20 09:45:29.897856,...,850862430,Sst-IRES-Cre,NaN,NaN,0,65.0,0.0,False,Sst_VISl_225,1


## Load ophys dataset for one experiment

Use VBA <code>get_ophys_dataset()</code> to get ophys data from NWB, from LIMS, or from LIMS + extended functionality 

In [7]:
help(loading.get_ophys_dataset)

Help on function get_ophys_dataset in module visual_behavior.data_access.loading:

get_ophys_dataset(ophys_experiment_id, include_invalid_rois=False, from_lims=False, from_nwb=False)
    Gets behavior + ophys data for one experiment (single imaging plane), either using the SDK LIMS API,
    SDK NWB API, or using BehaviorOphysDataset wrapper which inherits the LIMS API BehaviorOphysSession object,
    and adds functionality including invalid ROI filtering, extended stimulus_presentations and trials, and behavior movie data.
    
    Arguments:
        ophys_experiment_id {int} -- 9 digit ophys experiment ID
        include_invalid_rois {Boolean} -- if True, return all ROIs including invalid. If False, filter out invalid ROIs
        from_lims -- if True, loads dataset directly from BehaviorOphysSession.from_lims(). Invalid ROIs will be included.
        from_nwb -- if True, loads dataset directly from BehaviorOphysSession.from_nwb_path(). Invalid ROIs will not be included.
    
        

### preferred method - load from NWB files

Invalid ROIs are automatically excluded. 

In [8]:
experiment_id = experiments.index[50]
dataset = loading.get_ophys_dataset(experiment_id, from_nwb=True)

In [9]:
dataset.dff_traces.head()

,cell_roi_id,dff
cell_specimen_id,,
1086497784,1080727024,"[0.58022540807724, 0.23975388705730438, 0.3733..."
1086497908,1080727028,"[0.39847686886787415, 0.31532761454582214, 0.3..."
1086497034,1080727033,"[0.4359208941459656, 0.41052961349487305, 0.41..."
1086498071,1080727042,"[0.2510485351085663, 0.17865964770317078, 0.21..."
1086497438,1080727046,"[0.6064220070838928, 0.4349006712436676, 0.317..."


#### default behavior with no optional flags set will load from lims, filter out invalid ROIs, and provide access to face motion PCs and extended_stimulus_presentations 

In [10]:
experiment_id = experiments.index[50]
dataset = loading.get_ophys_dataset(experiment_id)

In [11]:
dataset.dff_traces.head()

,cell_roi_id,dff
cell_specimen_id,,
1086497784,1080727024,"[0.58022540807724, 0.23975388705730438, 0.3733..."
1086497908,1080727028,"[0.39847686886787415, 0.31532761454582214, 0.3..."
1086497034,1080727033,"[0.4359208941459656, 0.41052961349487305, 0.41..."
1086498071,1080727042,"[0.2510485351085663, 0.17865964770317078, 0.21..."
1086497438,1080727046,"[0.6064220070838928, 0.4349006712436676, 0.317..."


In [12]:
dataset.extended_stimulus_presentations.keys()

C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()
C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()
C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'cam1_exposure', 'cam2_exposure'}. Consider updating the sync file line labels.
  self._check_line_labels()


Index(['duration', 'end_frame', 'image_index', 'image_name', 'image_set',
       'index', 'omitted', 'start_frame', 'start_time', 'stop_time', 'change',
       'pre_change', 'epoch', 'mean_running_speed', 'mean_pupil_area', 'licks',
       'response_latency', 'response_binary', 'early_lick', 'rewards',
       'licked', 'lick_rate', 'rewarded', 'reward_rate', 'cropped_image_std',
       'warped_image_std', 'time_from_last_lick', 'time_from_last_reward',
       'time_from_last_change', 'time_from_last_omission',
       'flash_after_omitted', 'flash_after_change', 'image_name_next_flash',
       'image_index_next_flash', 'image_name_previous_flash',
       'image_index_previous_flash', 'lick_on_next_flash',
       'lick_rate_next_flash', 'lick_on_previous_flash',
       'lick_rate_previous_flash', 'running_rate', 'bout_rate', 'hit_bout',
       'hit_fraction', 'high_lick', 'high_reward', 'flash_metrics_epochs',
       'flash_metrics_labels', 'in_bout', 'bias', 'omissions1', 'task0',
     

## Get behavior experiments for mice in the March release

Use VBA <code>get_filtered_behavior_session_table()</code> with <code>release_data_only</code> flag set to True

In [13]:
help(loading.get_filtered_behavior_session_table)

Help on function get_filtered_behavior_session_table in module visual_behavior.data_access.loading:

get_filtered_behavior_session_table(release_data_only=True)
    Loads list of behavior sessions from SDK BehaviorProjectCache, and does some basic filtering and addition of columns, such as changing mouse_id from str to int and adding project code.
    
    Keyword Arguments:
        release_data_only {bool} -- If True, only return behavior sessions for mice with ophys data that will be included in the March data release.
        This does not include OPHYS_7_receptive_field_mapping or sessions with an unexpected session_type (i.e. where session_type is NaN)
    
    Returns:
        behavior_sessions -- Dataframe with behavior_session_id as the index and metadata as columns.



In [14]:
behavior_sessions = loading.get_filtered_behavior_session_table(release_data_only=True)
print('there are', len(behavior_sessions), 'behavior sessions in the release')

there are 3213 behavior sessions in the release


In [15]:
behavior_sessions.head()

,ophys_session_id,equipment_name,date_of_acquisition,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,session_type,project_code,has_passing_ophys_data
behavior_session_id,,,,,,,,,,,,,,
870987812,NaN,BEH.D-Box1,2019-05-17 14:46:33.550,850862423,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],F,80.0,68eaf198-92bb-4548-b5b4-da5ca03a8a68,TRAINING_0_gratings_autorewards_15min,VisualBehaviorMultiscope,False
872953842,NaN,BEH.D-Box1,2019-05-21 13:22:20.613,850862423,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],F,84.0,1e9c5671-a5a2-43f1-9d5e-ff7057a8547c,TRAINING_1_gratings,VisualBehaviorMultiscope,False
873774125,NaN,BEH.D-Box1,2019-05-22 13:19:52.005,850862423,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],F,85.0,4be18754-d341-4668-9551-ec732b20aca8,TRAINING_1_gratings,VisualBehaviorMultiscope,False
874790261,NaN,BEH.D-Box1,2019-05-23 13:40:31.405,850862423,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],F,86.0,b3b2c39b-b2a5-4e0a-88e9-2ce33518c02b,TRAINING_1_gratings,VisualBehaviorMultiscope,False
875207834,NaN,BEH.D-Box1,2019-05-24 11:06:39.332,850862423,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],F,87.0,2f5508f1-c65e-4254-b3f4-4a4be742356c,TRAINING_1_gratings,VisualBehaviorMultiscope,False


## Load data for one behavior session

Use VBA function <code>get_behavior_dataset()</code> to get behavior data from NWB or LIMS API

In [16]:
help(loading.get_behavior_dataset)

Help on function get_behavior_dataset in module visual_behavior.data_access.loading:

get_behavior_dataset(behavior_session_id, from_lims=False, from_nwb=False)
    Gets behavior data for one session, either using the SDK LIMS API, SDK NWB API, or using BehaviorDataset wrapper which inherits the LIMS API BehaviorSession object, and adds access to extended stimulus_presentations and trials.
    
    Arguments:
        behavior_session_id {int} -- 9 digit behavior session ID
        from_lims -- if True, loads dataset directly from BehaviorSession.from_lims()
        from_nwb -- if True, loads dataset directly from BehaviorSession.from_nwb_path(), after converting behavior_session_id to nwb_path via lims query
    
        If both from_lims and from_nwb are set to False, data will be loaded using the LIMS API then passed to the BehaviorDataset class which allows access to extended_stimulus_presentations and trials.
    
    Returns:
        object -- BehaviorSession or BehaviorDataset in

### preferred method - load from NWB files

In [17]:
behavior_session_id = behavior_sessions.index[100]

In [18]:
behavior_session = loading.get_behavior_dataset(behavior_session_id, from_nwb=True)

In [19]:
behavior_session.trials.head()

,start_time,stop_time,lick_times,reward_time,reward_volume,hit,false_alarm,miss,stimulus_change,aborted,...,catch,auto_rewarded,correct_reject,trial_length,response_time,change_frame,change_time,response_latency,initial_image_name,change_image_name
trials_id,,,,,,,,,,,,,,,,,,,,,
0,19.983123,21.651188,"[20.416797466110438, 20.616970813367516, 20.85...",NaN,0.0,False,False,False,False,True,...,False,False,False,1.668065,NaN,NaN,NaN,NaN,gratings_90,gratings_90
1,21.651188,25.521111,"[24.987340288236737, 25.204173311125487]",NaN,0.0,False,False,False,False,True,...,False,False,False,3.869923,NaN,NaN,NaN,NaN,gratings_90,gratings_90
2,25.521111,27.706278,[27.38996936613694],NaN,0.0,False,False,False,False,True,...,False,False,False,2.185167,NaN,NaN,NaN,NaN,gratings_90,gratings_90
3,27.706278,30.391897,"[29.85811912920326, 30.074969036504626]",NaN,0.0,False,False,False,False,True,...,False,False,False,2.685619,NaN,NaN,NaN,NaN,gratings_90,gratings_90
4,30.391897,33.527886,[33.21096881432459],NaN,0.0,False,False,False,False,True,...,False,False,False,3.135989,NaN,NaN,NaN,NaN,gratings_90,gratings_90


#### default behavior - load from lims, add support for extended_stimulus_presentations and extended_trials

In [20]:
behavior_dataset = loading.get_behavior_dataset(behavior_session_id)

In [21]:
behavior_dataset.trials

,lick_times,reward_time,reward_volume,hit,false_alarm,miss,stimulus_change,aborted,go,catch,...,correct_reject,start_time,stop_time,trial_length,response_time,change_frame,change_time,response_latency,initial_image_name,change_image_name
trials_id,,,,,,,,,,,,,,,,,,,,,
0,"[20.416797466110438, 20.616970813367516, 20.85...",NaN,0.00,False,False,False,False,True,False,False,...,False,19.983123,21.651188,1.668065,NaN,NaN,NaN,NaN,gratings_90,gratings_90
1,"[24.987340288236737, 25.204173311125487]",NaN,0.00,False,False,False,False,True,False,False,...,False,21.651188,25.521111,3.869923,NaN,NaN,NaN,NaN,gratings_90,gratings_90
2,[27.38996936613694],NaN,0.00,False,False,False,False,True,False,False,...,False,25.521111,27.706278,2.185167,NaN,NaN,NaN,NaN,gratings_90,gratings_90
3,"[29.85811912920326, 30.074969036504626]",NaN,0.00,False,False,False,False,True,False,False,...,False,27.706278,30.391897,2.685619,NaN,NaN,NaN,NaN,gratings_90,gratings_90
4,[33.21096881432459],NaN,0.00,False,False,False,False,True,False,False,...,False,30.391897,33.527886,3.135989,NaN,NaN,NaN,NaN,gratings_90,gratings_90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1536,"[3607.0470311967656, 3607.1971742017195, 3607....",NaN,0.00,False,False,False,False,True,False,False,...,False,3605.295568,3607.964482,2.668914,NaN,NaN,NaN,NaN,gratings_180,gratings_180
1537,[3610.666766759474],NaN,0.00,False,False,False,False,True,False,False,...,False,3607.964482,3610.983723,3.019241,NaN,NaN,NaN,NaN,gratings_180,gratings_180
1538,"[3612.7185027641244, 3613.002075134311]",NaN,0.00,False,False,False,False,True,False,False,...,False,3610.983723,3613.319009,2.335286,NaN,NaN,NaN,NaN,gratings_180,gratings_180


In [22]:
behavior_dataset.extended_stimulus_presentations.keys()

model outputs not available
model outputs not available


Index(['duration', 'end_frame', 'image_index', 'image_name', 'image_set',
       'index', 'omitted', 'orientation', 'phase', 'spatial_frequency',
       'start_frame', 'start_time', 'stop_time', 'change', 'pre_change',
       'epoch', 'mean_running_speed', 'licks', 'response_latency',
       'response_binary', 'early_lick', 'rewards', 'licked', 'lick_rate',
       'rewarded', 'reward_rate', 'cropped_image_std', 'warped_image_std',
       'time_from_last_lick', 'time_from_last_reward', 'time_from_last_change',
       'flash_after_change', 'image_name_next_flash', 'image_index_next_flash',
       'image_name_previous_flash', 'image_index_previous_flash',
       'lick_on_next_flash', 'lick_rate_next_flash', 'lick_on_previous_flash',
       'lick_rate_previous_flash'],
      dtype='object')